In [232]:
import requests
import json
import requests
import pandas as pd
from plotly import graph_objects as go
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [233]:
%run roadspending_gdp.ipynb
%run trips_duration.ipynb

Note: you may need to restart the kernel to use updated packages.


object


/var/folders/6f/26l7p7y90rd4sw3bc17185400000gn/T/ipykernel_44789/1627526132.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [234]:
def merging_data(final_gdp_roadspending_df, final_1plus_df):

    tourist_num = final_1plus_df
    tourist_num_copy = final_1plus_df.copy()
    gdp = final_gdp_roadspending_df[final_gdp_roadspending_df['Year'] > 2011]
    gdp_copy = final_gdp_roadspending_df[final_gdp_roadspending_df['Year'] > 2011].copy()


    min_trips = tourist_num['NumTrips'].min()
    max_trips = tourist_num['NumTrips'].max()

    min_gdp = gdp['GDP'].min()
    max_gdp = gdp['GDP'].max()

    tourist_num['Tourism St.'] = (tourist_num['NumTrips'] - min_trips) / (max_trips - min_trips)
    gdp['GDP St.'] = (gdp['GDP'] - min_gdp) / (max_gdp - min_gdp)

    tourist_num = pd.DataFrame(final_1plus_df, columns = ['Country', 'Year', 'Tourism St.'])
    gdp_actual = pd.DataFrame(gdp, columns = ["Country", "Year", "GDP St."])

    prediction_data = pd.merge(tourist_num, gdp_actual, on = ['Country', 'Year'])
    prediction_data["Year"] = prediction_data['Year'].astype(int)

    unfiltered_predicted_gdp =pd.DataFrame(gdp_copy, columns = ["Country", "Year", "GDP"])
    unfiltered_predicted_tourist =pd.DataFrame(tourist_num_copy, columns = ["Country", "Year", "NumTrips"])
    unfiltered_predicted_data = pd.merge(unfiltered_predicted_gdp, unfiltered_predicted_tourist, on = ['Country', 'Year'])

    return prediction_data, unfiltered_predicted_data




In [283]:
datas = merging_data(final_gdp_roadspending_df, final_1plus_df)
data = datas[0]
data_not_st = datas[1]
data_not_st

/var/folders/6f/26l7p7y90rd4sw3bc17185400000gn/T/ipykernel_44789/4017440539.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Year,GDP,NumTrips
0,Austria,2012,3.645983e+11,36003256
1,Austria,2013,3.828753e+11,36479782
2,Austria,2014,3.940021e+11,36681593
3,Austria,2015,3.399889e+11,35815124
4,Austria,2016,3.521979e+11,39365940
...,...,...,...,...
292,Sweden,2018,4.906677e+11,89740946
293,Sweden,2019,4.736304e+11,89854108
294,Sweden,2020,4.851817e+11,98845718
295,Sweden,2021,5.670964e+11,105262786


In [236]:
def get_lag_columns(country_df, lag = 3):

    country_df = country_df.sort_values('Year').reset_index(drop = True)
    country_df['Covid'] = country_df['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    country_df['GDP_lag'] = country_df['GDP St.'].shift(lag)
    country_df['Tourism_lag'] = country_df['Tourism St.'].shift(lag)
    country_df['Bias'] = 1

    return country_df.dropna().reset_index(drop = True)

In [237]:
def fit_regression(X, y):
    Xt = X.T
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.pinv(XtX)
    Xt_y = np.dot(Xt, y)
    beta = np.dot(XtX_inv, Xt_y)

    return beta

In [238]:
def predict_next_year(beta, X_new):
    return np.dot(beta, X_new)

In [239]:
def get_country_prediction(data, country, lag = 3):
    country_df0 = data[data['Country'] == country]
    country_df = get_lag_columns(country_df0, lag = lag)
    
    X = country_df[['Bias', 'GDP_lag', 'Tourism_lag']].values
    y_gdp = country_df['GDP St.'].values
    y_tourism = country_df['Tourism St.'].values

    beta_gdp = fit_regression(X, y_gdp)
    beta_tourism = fit_regression(X, y_tourism)

    last_year = country_df['Year'].max()
    next_year = last_year + 1
    lag_year = next_year - lag

    lag_row = country_df[country_df['Year'] == lag_year]
    
    GDP_lag_val = lag_row['GDP St.'].values[0]
    Tourism_lag_val = lag_row['Tourism St.'].values[0]

    X_new = np.array([1, GDP_lag_val, Tourism_lag_val])

    pred_gdp = predict_next_year(beta_gdp, X_new)
    pred_tourism = predict_next_year(beta_tourism, X_new)

    results_dict = {
        'Country': country, 
        'Predicted_GDP': pred_gdp,
        'Predicted_Tourism': pred_tourism,
        'Beta_GDP': beta_gdp,
        'Beta_Tourism': beta_tourism
    }

    return country_df

In [240]:
result = get_country_prediction(data, "France", lag=3)
print(result)


  Country  Year  Tourism St.   GDP St.  Covid   GDP_lag  Tourism_lag  Bias
0  France  2015     0.695409  0.538610      0  0.592176     0.771783     1
1  France  2016     0.682105  0.544586      0  0.621266     0.773372     1
2  France  2017     0.745108  0.570758      0  0.631417     0.782128     1
3  France  2018     0.744885  0.613899      0  0.538610     0.695409     1
4  France  2019     0.734383  0.600673      0  0.544586     0.682105     1
5  France  2020     0.579334  0.583564      1  0.570758     0.745108     1
6  France  2021     0.711278  0.654977      1  0.613899     0.744885     1
7  France  2022     0.827503  0.617366      0  0.600673     0.734383     1
8  France  2023     0.881959  0.673640      0  0.583564     0.579334     1


In [241]:
def get_country_prediction(data, country, lag=3):
    country_df0 = data[data['Country'] == country].copy()
    country_df = get_lag_columns(country_df0, lag=lag)

    X = country_df[['Bias', 'GDP_lag', 'Tourism_lag']].values
    y_gdp = country_df['GDP St.'].values
    y_tourism = country_df['Tourism St.'].values

    beta_gdp = fit_regression(X, y_gdp)
    beta_tourism = fit_regression(X, y_tourism)

    # Get the most recent year in the dataset
    last_year = country_df['Year'].max()

    # Get initial lag values
    lag_year = last_year - lag + 1
    lag_row = country_df[country_df['Year'] == lag_year]
    
    gdp_lag_val = lag_row['GDP St.'].values[0]
    tourism_lag_val = lag_row['Tourism St.'].values[0]

    # Store predictions for each future year
    predictions = []

    for i in range(1, lag + 1):
        year = last_year + i
        X_new = np.array([1, gdp_lag_val, tourism_lag_val])
        
        pred_gdp = predict_next_year(beta_gdp, X_new)
        pred_tourism = predict_next_year(beta_tourism, X_new)

        predictions.append({
            'Year': year,
            'Country': country,
            'Predicted_GDP': pred_gdp,
            'Predicted_Tourism': pred_tourism
        })

        # Update lag values for next step
        gdp_lag_val = pred_gdp
        tourism_lag_val = pred_tourism

    return predictions  # List of dicts for each predicted year


In [266]:
result = get_country_prediction(data, "Spain", lag=3)
result = pd.DataFrame(result)
result.sort_values(by=['Year'])
result

,Year,Country,Predicted_GDP,Predicted_Tourism
0,2024,Spain,0.308446,0.496448
1,2025,Spain,0.300261,0.475250
2,2026,Spain,0.303776,0.492371


In [296]:
def visualization(data, st_data): 
    min_trips = data['NumTrips'].min()
    max_trips = data['NumTrips'].max()

    min_gdp = data['GDP'].min()
    max_gdp = data['GDP'].max()

    st_data["Predicted_GDP_normal"] = st_data["Predicted_GDP"] * (max_gdp - min_gdp) + min_gdp
    st_data["Predicted_Tourism_normal"] = st_data["Predicted_Tourism"] * (max_trips - min_trips) + min_trips


    # Plot the line graph
    """    fig = px.line(st_data, x='Year', y='NumTrips', color='Country', color_discrete_sequence = px.colors.qualitative.Bold)
    fig.update_layout(title='Number of Trips (1 Plus Nights) per Year by Country',
                  xaxis_title='Year',
                  yaxis_title='Number of Trips',
                  legend_title='Country')"""
    
    country_name = st_data["Country"].iloc[0]

    matching_data = data[data["Country"] == country_name]

    merged = pd.concat([st_data, matching_data], ignore_index=True)

    merged = merged.sort_values(by="Year").reset_index(drop=True)

    merged = merged.drop(columns = ["Predicted_GDP", "Predicted_Tourism"])

    fig = go.Figure()

    # First trace → bottom x-axis (default)
    fig.add_trace(go.Scatter(
        x=merged["Year"], 
        y=merged["GDP"], 
        name='GDP',
        line=dict(color='firebrick', width=4),
        xaxis='x'
    ))
    fig.add_trace(go.Scatter(
        x=merged["Year"], 
        y=merged["Predicted_GDP_normal"], 
        name='GDP Predicted',
        line=dict(color='firebrick', width=4),
        xaxis='x'
    ))

    fig = go.Figure()

    # Right Y-Axis
    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['NumTrips'], 
        name='Tourist Numbers',
        yaxis='y2',
        line=dict(color='royalblue', width=4)
    ))
    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['Predicted_Tourism_normal'], 
        name='Tourist Numbers Predicted',
        yaxis='y2',
        line=dict(color='royalblue', width=4)
    ))

    # Layout with dual y-axes
    fig.update_layout(
        title='Predicted GDP and Tourism Numbers',
        xaxis=dict(title='Years'),
        yaxis=dict(
            title='GDP',
            titlefont=dict(color='firebrick'),
            tickfont=dict(color='firebrick')
        ),
        yaxis2=dict(
            title='Tourism Numbers',
            titlefont=dict(color='royalblue'),
            tickfont=dict(color='royalblue'),
            overlaying='y',
            side='right'
        )
    )
    
    fig.show()

In [297]:
visualization(data_not_st, result)

In [ ]:
def get_country_prediction(data, country):

    data['covid'] = data['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    data['Year'] = data['Year'].astype(int)
    country_data = data.sort_values(by = 'Year').reset_index(drop = True)

    lag = 3
    country_data['GDP_lag'] = country_data['GDP St.'].shift(lag)
    country_data['Tourism_lag'] = country_data['Tourism St.'].shift(lag)
    country_data['Bias'] = 1
    country_data = country_data.dropna().reset_index(drop = True)

    X = country_data[['Bias', 'GDP_lag', 'Tourism_lag']].values
    y_gdp = country_data['GDP St.'].values
    y_tourism = country_data['Tourism St.'].values

    Xt = X.T
    XtX_inv = np.linalg.pinv(np.matmul(Xt, X))
    beta_gdp = np.matmul(np.matmul(XtX_inv, Xt), y_gdp)
    beta_tourism = np.matmul(np.matmul(XtX_inv, Xt), y_tourism)




In [ ]:
get_country_prediction(data, country = 'Spain')

In [ ]:
def summary_stat(data):

    data['GDP_lag'] = data['GDP St.'].shift(1)
    data['Tourism_lag'] = data['Tourism St.'].shift(1)
    data['Bias'] = 1

    country_data = data.dropna().reset_index(drop=True)
    country_data = data.dropna().reset_index(drop=True)
    
    X = country_data[['Bias', 'GDP_lag', 'Tourism_lag']].values
    y_gdp = country_data['GDP St.'].values
    y_tourism = country_data['Tourism St.'].values
    
    m_gdp = np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y_gdp))
    m_tourism = np.matmul(np.linalg.inv(np.matmul(X.T, X)), np.matmul(X.T, y_tourism))

    np.set_printoptions(suppress=True)

    ypreds_tourism= np.matmul(X, m_tourism)
    ypreds_gdp = np.matmul(X, m_gdp)
    r2_gdp = r2_score(y_gdp, ypreds_gdp)
    r2_tourism = r2_score(y_tourism, ypreds_tourism)
    
    print(f'The r^2 prediction score for GDP is: {round(r2_gdp, 4)}')
    print(f'The r^2 prediction score for tourism is: {round(r2_tourism, 4)}')

In [ ]:
summary_stat(data)

The r^2 prediction score for GDP is: 0.8545
The r^2 prediction score for tourism is: 0.8294


In [ ]:
def train_test(data): 
    y_GDP = pd.array(data['GDP St.'])
    y_tourism = pd.array(data['Tourism St.'])
    X = pd.array(data['Year'])

# Split the data into training and testing sets
    X_train_gdp, X_test_gdp, y_train_gdp, y_test_gdp = train_test_split(X, y_GDP, test_size=0.3, random_state=42)
    X_train_tourism, X_test_tourism, y_train_tourism, y_test_tourism = train_test_split(X, y_tourism, test_size=0.3, random_state=42)

    return {"gdp": (X_train_gdp, X_test_gdp, y_train_gdp, y_test_gdp),
        "tourism": (X_train_tourism, X_test_tourism, y_train_tourism, y_test_tourism)}


In [ ]:
train_test(data)


{'gdp': (<NumpyExtensionArray>
  [2014, 2013, 2016, 2013, 2012, 2015, 2019, 2022, 2021, 2019,
   ...
   2015, 2022, 2013, 2022, 2020, 2020, 2023, 2022, 2018, 2018]
  Length: 207, dtype: int64,
  <NumpyExtensionArray>
  [2023, 2019, 2015, 2022, 2017, 2017, 2015, 2014, 2021, 2019, 2021, 2018, 2020,
   2017, 2018, 2018, 2019, 2021, 2023, 2016, 2017, 2021, 2012, 2022, 2014, 2021,
   2019, 2018, 2017, 2015, 2015, 2015, 2012, 2022, 2019, 2015, 2012, 2012, 2018,
   2013, 2020, 2018, 2017, 2023, 2015, 2016, 2013, 2017, 2020, 2016, 2021, 2020,
   2014, 2023, 2012, 2018, 2013, 2020, 2018, 2013, 2021, 2019, 2018, 2022, 2013,
   2020, 2013, 2018, 2012, 2022, 2016, 2017, 2013, 2019, 2016, 2018, 2020, 2015,
   2014, 2019, 2015, 2019, 2018, 2017, 2012, 2015, 2022, 2012, 2016, 2020]
  Length: 90, dtype: int64,
  <NumpyExtensionArray>
  [ 0.00883496299746336, 0.011063017984045222,  0.10322657337982327,
    0.03989336327390599,  0.04821342752335252, 0.007326792244996174,
    0.07433181040480649,   0.318

In [ ]:
visualization(data_not_st, forecast)

,Country,Year,Predicted_GDP,Predicted_Tourism,Predicted_GDP_normal,Predicted_Tourism_normal,GDP,NumTrips
0,Spain,2024,0.36,0.53,1.455931e+12,2.793708e+08,NaN,NaN
1,Spain,2025,0.36,0.52,1.455931e+12,2.741161e+08,NaN,NaN
2,Spain,2026,0.35,0.50,1.415726e+12,2.636069e+08,NaN,NaN
3,Spain,2027,0.34,0.49,1.375521e+12,2.583522e+08,NaN,NaN
4,Spain,2028,0.33,0.49,1.335317e+12,2.583522e+08,NaN,NaN
...,...,...,...,...,...,...,...,...
292,Sweden,2018,NaN,NaN,NaN,NaN,4.906677e+11,89740946.0
293,Sweden,2019,NaN,NaN,NaN,NaN,4.736304e+11,89854108.0
294,Sweden,2020,NaN,NaN,NaN,NaN,4.851817e+11,98845718.0
295,Sweden,2021,NaN,NaN,NaN,NaN,5.670964e+11,105262786.0
